In [ ]:
!pip install timm
!pip install adabelief-pytorch==0.2.0

This is a simple notebook to train the model. The idea is to understand and get upto speed with latest ideas. All experiment results will be logged on to this google doc. We are following this [notebook](https://www.kaggle.com/yasufuminakama/ranzcr-resnext50-32x4d-starter-training)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
import cv2
import matplotlib.pyplot as plt
import ast
import tqdm
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
path = '../input/ranzcr-clip-catheter-line-classification/'
train_annot = pd.read_csv(path+'train_annotations.csv')
train_annot.head()
train_images = os.listdir(path+'train')
train_labels = pd.read_csv(path+'train.csv')

In [ ]:
import ast
def plot_row(path,df_annotations,row=5):
    colors = [(228, 26, 28), (55, 126, 184), (152, 78, 163), (255, 255, 51), (166, 86, 40), (247, 129, 191), (153, 153, 153)]
    studyid = df_annotations.loc[row, 'StudyInstanceUID']
#     img = cv2.imread(df_annotations.loc[row, 'image_path'], 1)
    img_name = path+'train/'+studyid+'.jpg'
    img = cv2.imread(img_name,1)
    h,w,_ = img.shape
#     img = cv2.resize(img,(256,256))
    scale_x,scale_y = 256/h,256/w
    data = df_annotations[df_annotations['StudyInstanceUID'] == studyid]
    for index, row in data.reset_index().iterrows():
        print(row['label'])
        pts = np.array(ast.literal_eval(row['data']), np.int32)
        if row['label'][:3] == 'NGT':
            color_index = 0
        elif row['label'][:3] == 'CVC':
            color_index = 1
        elif row['label'][:3] == 'ETT':
            color_index = 3
        else:
            color_index = 4
        for i, pt in enumerate(pts):
            if i < len(pts)-1:
                a,b = pts[i]
#                 a = int(a*scale_x)
#                 b = int(b*scale_y)
                c,d = pts[i+1]
#                 c = int(c*scale_x)
#                 d = int(d*scale_y)
                cv2.line(img,tuple(pts[i]), tuple(pts[i+1]), colors[color_index], 30)
    plt.imshow(img)
    plt.show()


In [ ]:
plot_row(path,train_annot,5)

Most of the images are in the ball park of 2500x2500. They will be rescaled to smaller dimension to make training possible. The annotations would also be scaled down to make it possible. Also we dont have to work with the annotation. The annotation is an extra piece of information but its not necessary and is mostly used for our understanding. The entire game is based on image and multi label.  

In [ ]:
class CFG:
    debug=False
    print_freq=125
    num_workers=4
    model_name='resnest50d_1s4x24d'
    size=512
    scheduler='ReduceLROnPlateau'#['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=10
    factor=0.2 # ReduceLROnPlateau
    patience=4 # ReduceLROnPlateau
    eps=1e-6 # ReduceLROnPlateau
    T_max=6 # CosineAnnealingLR
    T_0=6 # CosineAnnealingWarmRestarts
    lr=1e-3
    min_lr=1e-6
    batch_size=32
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    target_size=11
    target_cols=['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal',
                 'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal', 
                 'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal',
                 'Swan Ganz Catheter Present']
    n_fold=2
    trn_fold=[0, 1]
    train=True
    
if CFG.debug:
    CFG.epochs = 1
    train = train.sample(n=100, random_state=CFG.seed).reset_index(drop=True)

In [ ]:
import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

from albumentations import (
    Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout, 
    IAAAdditiveGaussianNoise, Transpose, CLAHE,HueSaturationValue,JpegCompression,IAAPiecewiseAffine,IAASharpen
    ,Downscale
    )
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import timm
from adabelief_pytorch import AdaBelief

from torch.cuda.amp import autocast, GradScaler

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
OUTPUT_DIR='./'
def get_score(y_true, y_pred):
    scores = []
    for i in range(y_true.shape[1]):
        score = roc_auc_score(y_true[:,i], y_pred[:,i])
        scores.append(score)
    avg_score = np.mean(scores)
    return avg_score, scores

@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}]start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')

def init_logger(log_file=OUTPUT_DIR+'train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()


def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

### CV SPLIT

In [ ]:
folds = train_labels.copy()
Fold = GroupKFold(n_splits=CFG.n_fold)
groups = folds['PatientID'].values
for n, (train_index, val_index) in enumerate(Fold.split(folds,folds[CFG.target_cols],groups)):
    folds.loc[val_index,'fold'] = int(n)
folds['fold'] = folds['fold'].astype('int32')
folds.groupby('fold').size()

### DATASET

In [ ]:
class TrainDataset(Dataset):
    def __init__(self,df,transform=None):
        self.df = df
        self.file_names = df['StudyInstanceUID'].values
        self.labels = df[CFG.target_cols].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        file_name = self.file_names[idx]
        train_path = path+'/train'
#         print(file_name)
        file_path = f'{train_path}/{file_name}.jpg'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        label = torch.tensor(self.labels[idx]).float()
        return image,label
        

### Transforms

In [ ]:
# These are albumentation based and not the pytorch ones
def get_transforms(*,data):
    if data == 'train':
        return Compose([
            Resize(CFG.size,CFG.size),
            RandomResizedCrop(CFG.size,CFG.size,scale=(0.85,1.0)),
            HorizontalFlip(p=0.5),
            HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=0.7),
            CLAHE(clip_limit=(1,4), p=0.5),
            OneOf([
              JpegCompression(),
              Downscale(scale_min=0.1, scale_max=0.15),
              ], p=0.2),
              IAAPiecewiseAffine(p=0.2),
              IAASharpen(p=0.2),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    if data == 'valid':
        return Compose([
            Resize(CFG.size, CFG.size),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

In [ ]:
train_dataset = TrainDataset(train_labels,get_transforms(data='train'))
for i in range(3):
    image, label = train_dataset[3*i]
    plt.title(f'label: {label}')
    plt.imshow(image[0])
    plt.show()

### Model

In [ ]:
# class CustomModel(nn.Module):
#     def __init__(self,model,pretrained=False):
#         super().__init__()
#         self.model = timm.create_model(model,pretrained=pretrained,num_classes=CFG.target_size)
# #         n_features = self.model.fc.in_features
# #         self.model.fc = nn.Linear(n_features, CFG.target_size)
# #         n_features = self.model._fc.in_features
# #         self.model.fc = nn.Linear(n_features, CFG.target_size)
        
#     def forward(self,x):
#         x = self.model(x)
#         return x
class CustomModel(nn.Module):
    def __init__(self,model,pretrained=False):
        super().__init__()
        self.model = timm.create_model(model,pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        
        self.fc = nn.Linear(n_features, CFG.target_size)
#         n_features = self.model._fc.in_features
#         self.model.fc = nn.Linear(n_features, CFG.target_size)
        
    def forward(self,x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
#         x = self.model(x)
        return output

In [ ]:
class AverageMeter(object):
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        
    def update(self,val,n=1):
        self.val += val
        self.sum += val*n
        self.count += n
        self.avg = self.sum/self.count
        
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


In [ ]:
def train_epoch(train_loader,model,criterion,optimizer,epoch,scheduler,device):
#     GradScaler is a pytorch multiprecision construct. 
    scaler = GradScaler()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    
    model.train()
    start = end = time.time()
    global_step = 0
    for step, (images,labels) in enumerate(tqdm(train_loader)):
        data_time.update(time.time()-end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with autocast():
            y_preds = model(images)
            loss = criterion(y_preds,labels)
        losses.update(loss.item(),batch_size)
        if CFG.gradient_accumulation_steps >1 :
            loss /= CFG.gradient_accumulation_steps
        scaler.scale(loss).backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(),CFG.max_grad_norm)
        if (step+1)%CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
        batch_time.update(time.time()-end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  #'LR: {lr:.6f}  '
                  .format(
                   epoch+1, step, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(train_loader)),
                   grad_norm=grad_norm,
                   #lr=scheduler.get_lr()[0],
                   ))
    return losses.avg

def valid_epoch(valid_loader,model,criterion,device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    
    model.eval()
    preds = []
    start = end = time.time()
    for step,(images,labels) in enumerate(tqdm(valid_loader)):
        data_time.update(time.time()-end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        
        with torch.no_grad():
            y_preds = model(images)
        loss = criterion(y_preds,labels)
        losses.update(loss.item(),batch_size)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
        if CFG.gradient_accumulation_steps > 1:
            loss /= CFG.gradient_accumulation_steps
        batch_time.update(time.time()-end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(
                   step, len(valid_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(valid_loader)),
                   ))
    predictions = np.concatenate(preds)
    return losses.avg,predictions

### Train Loop

In [ ]:
def train_loop(folds,fold):
    LOGGER.info(f"========== fold: {fold} training ==========")
    train_idx = folds[folds['fold']!=fold].index
    val_idx = folds[folds['fold']==fold].index
#     print("val index",val_idx.shape)
    train_folds = folds.loc[train_idx].reset_index(drop=True)
    val_folds = folds.loc[val_idx].reset_index(drop=True)
#     print(val_folds.shape)
    val_labels = val_folds[CFG.target_cols].values
    
    train_dataset = TrainDataset(train_folds,transform=get_transforms(data='train'))
    val_dataset = TrainDataset(val_folds,transform=get_transforms(data='valid'))
    
    train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size,shuffle=True,num_workers = CFG.num_workers,pin_memory=True, drop_last=True)
    val_loader = DataLoader(val_dataset,batch_size=CFG.batch_size * 2,shuffle=False,num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    
    def get_scheduler(optimizer):
        if CFG.scheduler=='ReduceLROnPlateau':
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=CFG.factor, patience=CFG.patience, verbose=True, eps=CFG.eps)
        elif CFG.scheduler=='CosineAnnealingLR':
            scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)
        elif CFG.scheduler=='CosineAnnealingWarmRestarts':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=1, eta_min=CFG.min_lr, last_epoch=-1)
        return scheduler
    
    model = CustomModel(model=CFG.model_name,pretrained=True)
    model = model.to(device)
    
#     optimizer = Adam(model.parameters(),lr=CFG.lr,weight_decay=CFG.weight_decay,amsgrad=False)
    optimizer = AdaBelief(model.parameters(),lr=CFG.lr,eps=1e-16, betas=(0.9,0.999), weight_decouple = True, rectify = False)
    scheduler = get_scheduler(optimizer)
    
    criterion = nn.BCEWithLogitsLoss()
    
    best_score=0
    best_loss = np.inf
    
    for epoch in tqdm(range(CFG.epochs)):
        
        start_time = time.time()
        
        # train
        avg_loss = train_epoch(train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, preds = valid_epoch(val_loader, model, criterion, device)
        
        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(avg_val_loss)
        elif isinstance(scheduler, CosineAnnealingLR):
            scheduler.step()
        elif isinstance(scheduler, CosineAnnealingWarmRestarts):
            scheduler.step()
        
        # scoring
#         print(val_labels.shape,preds.shape,gt_labels.shape)
#         exit()
        score, scores = get_score(val_labels, preds)
        

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}  Scores: {np.round(scores, decimals=4)}')

        """
        if score > best_score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(), 
                        'preds': preds},
                        OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best.pth')
        """
        
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            LOGGER.info(f'Epoch {epoch+1} - Save Best Loss: {best_loss:.4f} Model')
            torch.save({'model': model.state_dict(), 
                        'preds': preds},
                        OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best.pth')
    
    check_point = torch.load(OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best.pth')
    for c in [f'pred_{c}' for c in CFG.target_cols]:
        val_folds[c] = np.nan
    val_folds[[f'pred_{c}' for c in CFG.target_cols]] = check_point['preds']

    return val_folds
    
    

In [ ]:
def main():

    """
    Prepare: 1.train  2.folds
    """

    def get_result(result_df):
        preds = result_df[[f'pred_{c}' for c in CFG.target_cols]].values
        labels = result_df[CFG.target_cols].values
        score, scores = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.4f}  Scores: {np.round(scores, decimals=4)}')
    
    if CFG.train:
        # train 
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(folds, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
        # CV result
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        # save result
        oof_df.to_csv(OUTPUT_DIR+'oof_df.csv', index=False)

In [ ]:
if __name__ == '__main__':
    main()